In [1]:
import numpy as np
import pandas as pd
from idtxl.bivariate_pid import BivariatePID
from idtxl.data import Data

from IPython.display import display, HTML

# Synergistic component of purely redundant signal

* Williams2010 and BROJA both find non-zero synergy in a purely redundant tripartite signal with added noise
* Synergy is not explained by random asymmetry in probability distribution, as a hand-crafted distribution with no asymmetry still produces nonzero synergy

### Get 3D probability distributions from data

In [2]:
# Generate N samples from a bernoulli probability distribution
def bernoulli(p, n):
    return (np.random.uniform(0, 1, n) < p).astype(int)

# Given 1D arrays of binary variables x,y,z,
# compute their joint probability distribution P(X,Y,Z)
def get_prob_cube(x,y,z):
    rez = np.zeros((2,2,2))
    for ix in [0, 1]:
        for iy in [0, 1]:
            for iz in [0, 1]:
                rez[ix, iy, iz] = np.mean(np.logical_and.reduce([x == ix, y==iy, z==iz]))
    return rez

In [3]:
x = bernoulli(0.5, 10000)
y = bernoulli(0.5, 10000)
z = bernoulli(0.5, 10000)

In [4]:
get_prob_cube(x,y,z)

array([[[0.1186, 0.1303],
        [0.126 , 0.1251]],

       [[0.1249, 0.1273],
        [0.1255, 0.1223]]])

### Generate noisy redundant data

However, ensure there is no randomness-induced asymmetry

In [5]:
kicker = 2
data2D = []
data2D += [[1,1,1]] * (73 * kicker)
data2D += [[1,1,0]] * (9 * kicker)
data2D += [[1,0,1]] * (9 * kicker)
data2D += [[1,0,0]] * (9 * kicker)
data2D += [[0,1,1]] * (9 * kicker)
data2D += [[0,1,0]] * (9 * kicker)
data2D += [[0,0,1]] * (9 * kicker)
data2D += [[0,0,0]] * (73 * kicker)
data2D = np.array(data2D).T

In [6]:
get_prob_cube(*data2D)

array([[[0.365, 0.045],
        [0.045, 0.045]],

       [[0.045, 0.045],
        [0.045, 0.365]]])

### Test IDTXL PID

So for purely symmetric data there is ZERO false positive uniques, but significant synergy

In [7]:
# Tartu PID estimator
def pid(dataPS):
    settings = {'pid_estimator': 'TartuPID', 'lags_pid': [0, 0]}

    dataIDTxl = Data(dataPS, dim_order='ps', normalise=False)
    pid = BivariatePID()
    rez = pid.analyse_single_target(settings=settings, data=dataIDTxl, target=2, sources=[0,1])
    return rez.get_single_target(2)

In [8]:
pid(data2D)

Adding data with properties: 3 processes, 400 samples, 1 replications
overwriting existing data

unq information s1: 0.00000000, s2: 0.00000000
shd information: 0.31992295, syn information: 0.09074957


{'num_err': (1.7751472514149214e-10, 0.0, 6.13204492738717e-09),
 'solver': 'ECOS http://www.embotech.com/ECOS',
 'shd_s1_s2': 0.3199229505088219,
 'syn_s1_s2': 0.09074957448659038,
 'unq_s1': 4.66498988739771e-09,
 'unq_s2': 4.664989887509928e-09,
 'source_1': [(0, 0)],
 'source_2': [(1, 0)],
 'selected_vars_sources': [(0, 0), (1, 0)],
 'current_value': (2, 0)}

## Compute PID using original Williams2010 formulation

In [9]:
# 1D entropy
def h(p):
    return -p.dot(np.log2(p))


# ND entropy
def hnd(p):
    return h(p.flatten())


# # Binary Mutual Info
# def mi_bin(pxy, px, py):
#     rez = 0
#     for ix in [0, 1]:
#         for iy in [0, 1]:
#             rez += pxy[ix, iy] * np.log2(pxy[ix, iy] / px[ix] / py[iy])
#     return rez

# # Binary Conditional Mutual Info
# def cmi_bin(pxyz, px, py, pz, pxz, pyz):
#     rez = 0
#     for ix in [0, 1]:
#         for iy in [0, 1]:
#             for iz in [0, 1]:
#                 rez += pxyz[ix, iy, iz] * np.log2(pxyz[ix, iy, iz] * pz[iz] / pxz[ix, iz] / pyz[iy, iz])
#     return rez


# Compute mutual informations for fixed value of py
def red_tmp(pxy, px, py):
    rezy = np.zeros(2)
    
    for ix in [0, 1]:
            rezy += (pxy[ix] / py) * np.log2(pxy[ix] / px[ix] / py)
    return rezy


# Compute redundancy as expected value of the minimal fixed value mutual information 
def red(px, py, pz, pxz, pyz):
    tmpXZ = red_tmp(pxz, px, pz)
    tmpYZ = red_tmp(pyz, py, pz)
    tmpMinZ = np.min([tmpXZ, tmpYZ], axis=0)
    
    return pz.dot(tmpMinZ)
    

def naive_pid(dataPS):
    # PMF
    x,y,z = dataPS
    pXYZ = get_prob_cube(x,y,z)
    
    # Marginals
    pX = np.sum(pXYZ, axis=(1,2))
    pY = np.sum(pXYZ, axis=(0,2))
    pZ = np.sum(pXYZ, axis=(0,1))
    pXY = np.sum(pXYZ, axis=2)
    pXZ = np.sum(pXYZ, axis=1)
    pYZ = np.sum(pXYZ, axis=0)
    
    # Entropies
    hX = h(pX)
    hY = h(pY)
    hZ = h(pZ)
    hXY = hnd(pXY)
    hXZ = hnd(pXZ)
    hYZ = hnd(pYZ)
    hXYZ = hnd(pXYZ)
    
    df = pd.DataFrame()
    df['Entropy Measures'] = ['H(X)', 'H(Y)', 'H(Z)', 'H(XY)', 'H(XZ)', 'H(YZ)', 'H(XYZ)']
    df['Entropy Values'] = [hX, hY, hZ, hXY, hXZ, hYZ, hXYZ]
    
    # Mutual Informations
    miXY = hX + hY - hXY
    miXZ = hX + hZ - hXZ
    miYZ = hY + hZ - hYZ
    miXYZ = hXY + hZ - hXYZ
    miXZY = hXZ + hY - hXYZ
    miYZX = hYZ + hX - hXYZ
    cmiXYZ = hXZ + hYZ - hXYZ - hZ
    cmiXZY = hXY + hYZ - hXYZ - hY
    cmiYZX = hXY + hXZ - hXYZ - hX
    
#     miXY = mi_bin(pXY, pX, pY)
#     miXZ = mi_bin(pXZ, pX, pZ)
#     miYZ = mi_bin(pYZ, pY, pZ)
#     cmiXYZ = cmi_bin(pXYZ,                      pX, pY, pZ, pXZ,   pYZ)
#     cmiXZY = cmi_bin(pXYZ.transpose((0, 2, 1)), pX, pZ, pY, pXY,   pYZ.T)
#     cmiYZX = cmi_bin(pXYZ.transpose((1, 2, 0)), pY, pZ, pX, pXY.T, pXZ.T)
    
    df2 = pd.DataFrame()
    df2['MI Measures'] = [
        'I(X:Y)', 'I(X:Z)', 'I(Y:Z)',
        'I(XY:Z)', 'I(XZ:Y)', 'I(YZ:X)',
        'I(X:Y|Z)', 'I(X:Z|Y)', 'I(Y:Z|X)'
    ]
    df2['MI Values']   = [
        miXY, miXZ, miYZ,
        miXYZ, miXZY, miYZX,
        cmiXYZ, cmiXZY, cmiYZX
    ]
    
    # PID
    redXYZ = red(pX, pY, pZ, pXZ, pYZ)
    unqXZ = miXZ - redXYZ
    unqYZ = miYZ - redXYZ
    synXYZ = cmiXZY - unqXZ
    
    df3 = pd.DataFrame()
    df3['PID Measures'] = ['R(X:Y:Z)', 'U(X:Z|Y)', 'U(Y:Z|X)', 'S(X:Y:Z)']
    df3['PID Values']   = [redXYZ, unqXZ, unqYZ, synXYZ]
    
    display(pd.concat([df, df2, df3], axis=1).fillna(''))

In [10]:
naive_pid(data2D)

,Entropy Measures,Entropy Values,MI Measures,MI Values,PID Measures,PID Values
0,H(X),1.0,I(X:Y),0.319923,R(X:Y:Z),0.319923
1,H(Y),1.0,I(X:Z),0.319923,U(X:Z|Y),0.0
2,H(Z),1.0,I(Y:Z),0.319923,U(Y:Z|X),0.0
3,H(XY),1.680077,I(XY:Z),0.410673,S(X:Y:Z),0.09075
4,H(XZ),1.680077,I(XZ:Y),0.410673,,
5,H(YZ),1.680077,I(YZ:X),0.410673,,
6,H(XYZ),2.269405,I(X:Y|Z),0.090750,,
7,,,I(X:Z|Y),0.090750,,
8,,,I(Y:Z|X),0.090750,,
